In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install nltk
!pip install torch
!pip install numpy
!pip install sacrebleu
!pip install rouge_score
!pip install bert_score
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b2506d383216e53833e7d6060e2698fb6cad44b7e65bc4ff8956d12fead02337
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True
max_seq_length = 512
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Jiten1024/Mistral_7b_test_ep_5_r_8_a_64", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Mistral patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Unsloth 2024.12.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096, padding_idx=770)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an interviewer who asks a Java question to the user based on topic, subtopic, and difficulty.

### Input:
Topic: {topic}
Subtopic: {subtopic}
Difficulty: {difficulty}

### Response:
{question}
"""

inputs = tokenizer(
[
    prompt.format(
        topic="Java Variables",
        subtopic="Variable Declaration",
        difficulty="Easy",
        question=""  # Leave blank for generation
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are an interviewer who asks a Java question to the user based on topic, subtopic, and difficulty.\n\n### Input:\nTopic: Java Variables\nSubtopic: Variable Declaration\nDifficulty: Easy\n\n### Response:\n\nWhat is the purpose of declaring a variable in Java?\n</s>']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from collections import defaultdict

nltk.download('punkt')
# Load the dataset
df = pd.read_csv('/content/drive/My Drive/pass_QA_topic_stopic_diff (1).csv')
# Group questions by unique combinations of Topic, Sub-Topic, Passage, and Difficulty
grouped_questions = defaultdict(list)
for _, row in df.iterrows():
    key = (row['Topic'], row['Sub-Topic'], row['Difficulty'])
    grouped_questions[key].append(row['Question'])

# Convert the grouped questions to a DataFrame
expanded_rows = []
for (topic, sub_topic, difficulty), questions in grouped_questions.items():
    expanded_rows.append({
        'Topic': topic,
        'Sub-Topic': sub_topic,
        'Difficulty': difficulty,
        'All_Questions': questions  # Keep as list, or convert to string if needed
    })

df_expanded = pd.DataFrame(expanded_rows)

# If you want to join the questions into a single string (optional)
# df_expanded['All_Questions'] = df_expanded['All_Questions'].apply(lambda x: ' | '.join(x))

print("Checkpoint 1")
print(df_expanded.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Checkpoint 1
            Topic                                Sub-Topic Difficulty  \
0  Java Variables  Variable Declaration and Initialization       Easy   
1  Java Variables  Variable Declaration and Initialization     Medium   
2  Java Variables  Variable Declaration and Initialization       Hard   
3  Java Variables                           Variable Types       Easy   
4  Java Variables                           Variable Types     Medium   

                                       All_Questions  
0  [What is the purpose of declaring a variable i...  
1  [What are some examples of variable types in J...  
2  [Explain the process of declaring and initiali...  
3  [What are the two types of variables supported...  
4  [What are some examples of reference types in ...  


In [ ]:
df2 = df_expanded

In [ ]:
# Function to generate a question and extract the relevant part
def generate_question_extracted(topic, subtopic, difficulty):
    prompt = (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\nYou are an interviewer who asks a Java question to the user based on topic, subtopic, and difficulty.\n\n"
        f"### Input:\nTopic: {topic}\nSubtopic: {subtopic}\nDifficulty: {difficulty}\n\n### Response:\n"
    )

    inputs = tokenizer(
        [prompt], return_tensors="pt", padding=True, truncation=True
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract question after '### Response:'
    question = decoded_output.split("### Response:")[1].strip()
    return question

# Generate and add questions to the DataFrame
df2['Generated_Question'] = df2.apply(
    lambda row: generate_question_extracted(row['Topic'], row['Sub-Topic'], row['Difficulty']), axis=1
)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# Function to calculate scores
def calculate_scores(row):
    generated_question = row['Generated_Question']
    references = row['All_Questions']  # Assuming this is a list of reference questions

    # BLEU Scores
    candidate = generated_question.split()
    ref_tokens = [ref.split() for ref in references]
    smoothing = SmoothingFunction().method1
    row['BLEU-1'] = sentence_bleu(ref_tokens, candidate, weights=(1, 0, 0, 0), smoothing_function=smoothing)
    row['BLEU-2'] = sentence_bleu(ref_tokens, candidate, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
    row['BLEU-3'] = sentence_bleu(ref_tokens, candidate, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing)
    row['BLEU-4'] = sentence_bleu(ref_tokens, candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)

    # ROUGE Scores
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge.score(generated_question, ' '.join(references))
    row['ROUGE-1'] = rouge_scores['rouge1'].fmeasure
    row['ROUGE-2'] = rouge_scores['rouge2'].fmeasure
    row['ROUGE-L'] = rouge_scores['rougeL'].fmeasure

    # BERT Scores
    P, R, F1 = bert_score([generated_question], [' '.join(references)], lang='en')
    row['BERT-Precision'] = P.item()
    row['BERT-Recall'] = R.item()
    row['BERT-F1'] = F1.item()

    return row

# Apply scoring function
df_scores2 = df2.apply(calculate_scores, axis=1)

# Save and display results



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [ ]:
df_scores2.to_csv('scores_output.csv', index=False)
print(df_scores2[['Topic', 'Sub-Topic', 'Difficulty', 'Generated_Question', 'BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BERT-Precision', 'BERT-Recall', 'BERT-F1']].head())

            Topic                                Sub-Topic Difficulty  \
0  Java Variables  Variable Declaration and Initialization       Easy   
1  Java Variables  Variable Declaration and Initialization     Medium   
2  Java Variables  Variable Declaration and Initialization       Hard   
3  Java Variables                           Variable Types       Easy   
4  Java Variables                           Variable Types     Medium   

                                  Generated_Question    BLEU-1   BLEU-2  \
0  What is the purpose of declaring a variable in...  1.000000  1.00000   
1  What happens if a variable is used without bei...  1.000000  1.00000   
2  Can a variable be declared and initialized at ...  0.461538  0.27735   
3  What are the two types of variables supported ...  1.000000  1.00000   
4  What are some examples of reference types in J...  1.000000  1.00000   

     BLEU-3    BLEU-4   ROUGE-1   ROUGE-2   ROUGE-L  BERT-Precision  \
0  1.000000  1.000000  0.512821  0.4864